In [1]:
import Modules.Model.Segment as Segment
import Modules.Model.Detector as Detector
import Modules.Model.Classifier as Classifier
from torchvision.transforms._presets import ObjectDetection
import os
from Modules.dataloader import base_dataset, DataModule

c:\Users\197796\Anaconda3\envs\AISeed\lib\site-packages\neptune\common\warnings.py:62: NeptuneDeprecationWarning: You're importing the Neptune client library via the deprecated `neptune.new` module, which will be removed in a future release. Import directly from `neptune` instead.
  warnings.warn(


In [2]:
# class PennFudanDataset(base_dataset):

#     def __init__(self, transform=None,
#                  data_path='../../data/PennFudanPed/'):
        
        
#         imgs = list(map(lambda img: os.path.join(data_path, "PNGImages", img), 
#                         sorted(os.listdir(os.path.join(data_path, "PNGImages")))))
#         masks = list(map(lambda mask: os.path.join(data_path, "PedMasks", mask), 
#                         sorted(os.listdir(os.path.join(data_path, "PedMasks")))))
        
#         if(transform is None):
#             transform = partial(ObjectDetection)()

#         super(PennFudanDataset, self).__init__(data=imgs, targets=masks, transform=transform)

#     def __parse_mask__(self, mask):
#         mask = np.array(mask)
#         # instances are encoded as different colors
#         obj_ids = np.unique(mask)
#         # first id is the background, so remove it
#         obj_ids = obj_ids[1:]

#         # split the color-encoded mask into a set
#         # of binary masks
#         masks = mask == obj_ids[:, None, None]

#         # get bounding box coordinates for each mask
#         num_objs = len(obj_ids)
#         boxes = []
#         for i in range(num_objs):
#             pos = np.nonzero(masks[i])
#             xmin = np.min(pos[1])
#             xmax = np.max(pos[1])
#             ymin = np.min(pos[0])
#             ymax = np.max(pos[0])
#             boxes.append([xmin, ymin, xmax, ymax])
        
#         return boxes, masks, obj_ids

#     def __load_sample__(self, index) -> dict:
        
#         img = Image.open(self.data[index]).convert("RGB")
#         mask = Image.open(self.targets[index]).convert('L')
#         boxes, masks, obj_ids = self.__parse_mask__(mask)

#         return {'data': img,
#                 'target': {'boxes': boxes,
#                            'masks': masks,
#                            'obj_ids': obj_ids},
#                 'original': img}
    
#     def __transform__(self, sample: dict) -> dict:
        
#         boxes = sample['target']['boxes']
#         masks = sample['target']['masks']
#         num_objs = len(sample['target']['obj_ids'])

#         # convert everything into a torch.Tensor
#         boxes = torch.as_tensor(boxes, dtype=torch.float32)
#         # there is only one class
#         labels = torch.ones((num_objs,), dtype=torch.int64)
#         masks = torch.as_tensor(masks, dtype=torch.uint8)

#         # image_id = torch.tensor([index])
#         area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
#         # suppose all instances are not crowd
#         iscrowd = torch.zeros((num_objs,), dtype=torch.int64)

#         target = {}
#         target["boxes"] = boxes
#         target["labels"] = labels
#         target["masks"] = masks
#         target["image_id"] = torch.tensor([0])
#         target["area"] = area
#         target["iscrowd"] = iscrowd

#         sample.update({'target': target})

#         return super().__transform__(sample)

In [3]:
from torchvision import datasets
from torchvision import transforms

class CIFAR10(base_dataset):
    
    def __init__(self, data_path, train, transform=None) -> None:

        dataset = datasets.CIFAR10(root=data_path, 
                                         train=train, 
                                         transform=transform, download=True)
        
        if transform is None:
            transform = transforms.Compose([
                    transforms.ToPILImage(),  # Convert numpy array to PIL Image
                    transforms.Resize((224, 224)),  # Resize the image to 32x32
                    transforms.ToTensor(),  # Convert PIL Image to PyTorch Tensor
                    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize the data
                ])

        super().__init__(data=dataset.data[:100], targets=dataset.targets[:100], transform=transform)
    

In [4]:
import yaml
from Modules.wrap_model import WrapModel
from Modules.ultis import get_trainer, seed_everything, get_class_overlay, get_basic_callbacks, get_metrics, get_optimizer, get_loss_function
import torchvision
import torch
from pytorch_lightning.loggers import NeptuneLogger

seed_everything(44)
torch.backends.cudnn.benchmark = True # keep True if all the input have same size.
torch.set_float32_matmul_precision('medium')


with open('config/new-config.yml', 'r') as stream:
    PARAMS = yaml.safe_load(stream)

logger = NeptuneLogger(
        project=PARAMS['logger']['project'],
        # api_key=PARAMS['logger']['api_key'],
        tags=PARAMS['logger']['tags'],
        log_model_checkpoints=False)

#load data
train_dataset = CIFAR10(data_path=PARAMS['dataset_settings']['path'], train=True)
test_dataset = CIFAR10(data_path=PARAMS['dataset_settings']['path'], train=False)

data_loader = DataModule(train_dataset,
                        test_dataset,
                        batch_size=PARAMS['dataset_settings']['batch_size'],
                        num_workers=PARAMS['dataset_settings']['num_workers'])

classification_model = Classifier.ClassificationModel(model=torchvision.models.resnet18,
                                                      weight=torchvision.models.ResNet18_Weights.DEFAULT,
                                                      is_freeze=PARAMS['architect_settings']['is_freeze'],
                                                      is_full=PARAMS['architect_settings']['is_full'],
                                                      n_cls=PARAMS['dataset_settings']['n_cls'])
                                                      
# create model
model = WrapModel(  model=classification_model,
                    optimizer_fn=get_optimizer(PARAMS['training_settings']['optimizer'],
                                            PARAMS['training_settings']['lr']),
                    loss_fn=get_loss_function(PARAMS['training_settings']['loss']),
                    train_metrics_fn=get_metrics(PARAMS['training_settings']['metrics'],
                                        PARAMS['dataset_settings']['n_cls'],
                                        prefix='train_'),
                    val_metrics_fn=get_metrics(PARAMS['training_settings']['metrics'],
                                        PARAMS['dataset_settings']['n_cls'],
                                        prefix='val_'),
                    log_fn=get_class_overlay
                  )

trainer = get_trainer(logger=logger,
                      gpu_ids=PARAMS['training_settings']['gpu_ids'],
                      monitor=PARAMS['training_settings']['monitor'],
                      mode=PARAMS['training_settings']['mode'],
                      max_epochs=PARAMS['training_settings']['max_epochs'],
                      ckpt_path=PARAMS['training_settings']['ckpt_path'],
                      early_stopping=PARAMS['training_settings']['early_stopping'])

# train
trainer.fit(model, data_loader)
# test
# trainer.test(model, data_loader)

Files already downloaded and verified
Files already downloaded and verified


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
c:\Users\197796\Anaconda3\envs\AISeed\lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:612: UserWarning: Checkpoint directory E:\code\AISeed\Pytorch-pretrained-models\models\test exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name             | Type                | Params
---------------------------------------------------------
0 | model            | ClassificationModel | 11.2 M
1 | loss_fn          | CrossEntropyLoss    | 0     
2 | train_metrics_fn | MetricColl

Sanity Checking: 0it [00:00, ?it/s]

c:\Users\197796\Anaconda3\envs\AISeed\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:430: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 24 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
c:\Users\197796\Anaconda3\envs\AISeed\lib\site-packages\neptune\common\warnings.py:62: NeptuneWarning: To avoid unintended consumption of logging hours during interactive sessions, the following monitoring options are disabled unless set to 'True' when initializing the run: 'capture_stdout', 'capture_stderr', and 'capture_hardware_metrics'.
  warnings.warn(


https://app.neptune.ai/kaori/AISeed/e/AIS-263


...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=5` reached.
